In [64]:
import openai
import faiss
import numpy as np
from dotenv import load_dotenv

load_dotenv()

True

In [65]:
dimension = 1536  # Dimension of the OpenAI embeddings
index = faiss.IndexFlatL2(dimension)
embedding_metadata = []

def embed_text(text):
    response = openai.embeddings.create(
        input=text, 
        model="text-embedding-ada-002"
    )
    return response.data[0].embedding

In [67]:
def update_vector_store(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    
    # Split content into smaller chunks if necessary
    chunks = content.split('-'*20)  # Split by lines for simplicity; can use more sophisticated chunking
    
    for i, chunk in enumerate(chunks):
        embedding = embed_text(chunk)
        embedding_np = np.array(embedding).astype('float32')
        index.add(np.array([embedding_np]))
        embedding_metadata.append({'id': f'doc-{i}', 'text': chunk})

# Update the vector store with the current text file
update_vector_store('test.txt')

In [74]:
def query_vector_store(query, top_k=5):
    query_embedding = embed_text(query)
    query_embedding_np = np.array([query_embedding]).astype('float32')
    distances, indices = index.search(query_embedding_np, top_k)
    return [embedding_metadata[idx] for idx in indices[0]]

def generate_response(query):
    matches = query_vector_store(query)
    context = " ".join([match['text'] for match in matches])

    response = openai.chat.completions.create(
            model='gpt-4o-mini',
            messages=[
                {"role": "system", "content": "Your name is Memoro and you are a memory assistant listening to my conversations."},
                {"role": "user", "content": context},
                {"role": "user", "content": query}
            ]
        )
    return response.choices[0].message.content

query = 'Did I have any arguments recently?'
generate_response(query)


'Yes, you had a debate with someone named John about the merits of working remotely versus being in the office. You expressed concerns regarding team camaraderie and face-to-face interactions, while John highlighted the benefits of flexibility and work-life balance that remote work can offer. It seems you both had differing opinions on the topic.'